<a href="https://colab.research.google.com/github/douglascsantos/colab_analyse_text/blob/main/api_pubmed_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>